<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab05/expansao_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import pandas as pnd
import numpy as npy
import nltk
import re
import collections
import heapq
import time
import bisect
import math
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
resultado = pnd.read_csv("results.csv")

[nltk_data] Downloading package stopwords to C:\Users\Rubens
[nltk_data]     Batista\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [187]:
tokenizer = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
n = 0
rank = {}

for texto in resultado.text:
  palavras = [palavra for palavra in tokenizer.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra)) and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in rank.keys():
      rank[t] = []
    rank[t].append(n)
    
for elemento in rank.items():
  d = dict(collections.Counter(elemento[1]))
  rank[elemento[0]] = list(d.items())

Palavras que terão sua consulta expandida:

In [188]:
consultas = ["presidente", "governo", "país", "pessoas", "mulheres"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [189]:
def n_documentos_palavra(palavra, ranking):
  return len(ranking[palavra])

def n_documentos_ambas(a, b, ranking):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n
  
def n_documentos():
  return resultado.text.count()

In [190]:
def calcula_MIM(a, b, ranking):
  n_a = n_documentos_palavra(a, ranking)
  n_b = n_documentos_palavra(b, ranking)
  n_ab = n_documentos_ambas(a, b, ranking)
  quo = (n_a * n_b)
  mim = n_ab / quo if quo != 0 else 0
  return (mim,b)
  
def calcula_EMIM(a, b, ranking, n):
  n_a = n_documentos_palavra(a, ranking)
  n_b = n_documentos_palavra(b, ranking)
  n_ab = n_documentos_ambas(a, b, ranking)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * npy.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calcula_X(a, b, ranking, n):
  n_a = n_documentos_palavra(a, ranking)
  n_b = n_documentos_palavra(b, ranking)
  n_ab = n_documentos_ambas(a, b, ranking)
  quo = (n_a * n_b)
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / quo if quo != 0 else 0
  return (x,b)
  
def calcula_DICE(a, b, ranking):
  n_a = n_documentos_palavra(a, ranking)
  n_b = n_documentos_palavra(b, ranking)
  n_ab = n_documentos_ambas(a, b, ranking)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

In [191]:
def cria_tabela_de_metricas(consulta, rank, n):
  tabela_top_10 = pnd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for palavra in rank:
    if palavra != consulta:
      bisect.insort(MIM, calcula_MIM(consulta, palavra, rank))
      bisect.insort(EMIM,calcula_EMIM(consulta, palavra, rank, n))
      bisect.insort(X,calcula_X(consulta, palavra, rank, n))
      bisect.insort(Dice,calcula_DICE(consulta, palavra, rank))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  tabela_top_10['MIM'] = [palavra[1] for palavra in MIM[:10]]
  tabela_top_10['EMIM'] = [palavra[1] for palavra in EMIM[:10]]
  tabela_top_10['X²'] = [palavra[1] for palavra in X[:10]]
  tabela_top_10['Dice'] = [palavra[1] for palavra in Dice[:10]]
  
  tabela_top_10.index += 1
  
  return tabela_top_10

In [192]:
n = resultado.text.count()
tabela_01 = cria_tabela_de_metricas(consultas[0], rank, n)
tabela_02 = cria_tabela_de_metricas(consultas[1], rank, n)
tabela_03 = cria_tabela_de_metricas(consultas[2], rank, n)
tabela_04 = cria_tabela_de_metricas(consultas[3], rank, n)
tabela_05 = cria_tabela_de_metricas(consultas[4], rank, n)

####1.1 Top-10 palavras associadas a 'presidente' 

In [193]:
tabela_01

,MIM,EMIM,X²,Dice
1,útil,bolsonaro,técnicos,bolsonaro
2,úteis,disse,consumidor,disse
3,último,jair,agência,brasil
4,última,lula,vem,jair
5,ótimo,prisão,governos,governo
6,ótima,silva,repasse,feira
7,órgão,livre,necessários,nesta
8,ódio,dias,informa,contra
9,índios,inácio,funciona,ser
10,índio,república,famílias,sobre


#### 1.2 Top-10 palavras associadas a 'governo'

In [194]:
tabela_02

,MIM,EMIM,X²,Dice
1,única,bolsonaro,sendo,bolsonaro
2,último,afirmou,lado,presidente
3,ótimo,ainda,desenvolvimento,brasil
4,ótima,sendo,vou,jair
5,órgão,lado,chefe,disse
6,índios,jair,povo,afirmou
7,étnica,povo,sim,paulo
8,ética,paulo,ainda,ainda
9,época,país,secretário,país
10,ânimo,vai,publicada,ser


#### 1.3 Top-10 palavras associadas a 'país'

In [195]:
tabela_03

,MIM,EMIM,X²,Dice
1,útil,brasil,setor,brasil
2,étnicas,mundo,méxico,governo
3,ética,países,ato,nacional
4,árbitro,nacional,saída,paulo
5,álvaro,setor,importância,mundo
6,álcool,grande,declarou,grande
7,áfrica,ato,mundo,pessoas
8,zelândia,fernando,visita,bolsonaro
9,yeend,eua,oposição,países
10,yad,méxico,países,afirmou


#### 1.4 Top-10 palavras associadas a 'pessoas'

In [196]:
tabela_04

,MIM,EMIM,X²,Dice
1,ótimo,então,então,então
2,ótima,todo,seguinte,país
3,ídolo,deputado,facebook,paulo
4,étnica,país,veio,todo
5,ética,antes,retorno,deputado
6,ânimos,seguinte,procurando,federal
7,âmago,facebook,parece,dias
8,árbitro,realidade,hora,disse
9,álvaro,dentro,homens,onde
10,álcool,sempre,escrito,apenas


#### 1.5 Top-10 palavras associadas a 'mulheres'

In [197]:
tabela_05

,MIM,EMIM,X²,Dice
1,volume,seguro,seguro,seguro
2,verdadeiramente,forte,forte,forte
3,velhice,economista,economista,economista
4,variáveis,aposentadoria,aposentadoria,aposentadoria
5,trouxeram,igual,volume,igual
6,tramitação,homens,verdadeiramente,homens
7,timing,aprovada,velhice,aprovada
8,tensões,problema,variáveis,volume
9,tecnicamente,banco,trouxeram,verdadeiramente
10,tecido,porque,tramitação,velhice


#### Métricas com resultados satisfatórios:
EMIM, X², Dice.

Essas mostraram resultados mais próximos ao contexto da palavra que foi utilizada. MIM mostrou resultados mais distantes do contexto. Dice foi a de melhores resultados.

### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

#### Algortimo : Termo-por-vez

In [198]:
def term_a_time(consulta, indices, k):    
    a = {} # Accumulator HashTable
    indices_consultas = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for palavra in consulta.split(" "):
      if palavra in indices.keys():
          indices_consultas.append(indices[palavra])
    
    for ind in indices_consultas:
      for i in ind:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    return heapq.nlargest(k, rank)

In [199]:
def top_docs(consulta, k):
  
  top_docs = [doc for score,doc in term_a_time(consulta, rank, k)]
  top_docs_ranking = {} 
  
  for palavra in rank:
    top_docs_ranking[palavra] = []
    for doc_score in rank[palavra]:
      if doc_score[0] in top_docs:
        top_docs_ranking[palavra].append(doc_score)
        
  return top_docs_ranking

#### Top-3 Documentos


In [200]:
k = 3
rank_d1 = top_docs(consultas[0], k)
rank_d2 = top_docs(consultas[1], k)
rank_d3 = top_docs(consultas[2], k)
rank_d4 = top_docs(consultas[3], k)
rank_d5 = top_docs(consultas[4], k)

In [201]:
tabela_top1 = cria_tabela_de_metricas(consultas[0], rank_d1, k)
tabela_top2 = cria_tabela_de_metricas(consultas[1], rank_d2, k)
tabela_top3 = cria_tabela_de_metricas(consultas[2], rank_d3, k)
tabela_top4 = cria_tabela_de_metricas(consultas[3], rank_d4, k)
tabela_top5 = cria_tabela_de_metricas(consultas[4], rank_d5, k)

##### Top-10 palavras associadas aos Top-3 Documentos de 'presidente'

In [202]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_01['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top1['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,turno
2,disse,sobre
3,brasil,sido
4,jair,ser
5,governo,segundo
6,feira,qualquer
7,nesta,país
8,contra,paulo
9,ser,ministério
10,sobre,lembrar


##### Top-10 palavras associadas aos Top-3 Documentos de 'governo'

In [203]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_02['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top2['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,parte
2,presidente,boa
3,brasil,três
4,jair,todos
5,disse,todo
6,afirmou,ser
7,paulo,saúde
8,ainda,responsável
9,país,reformas
10,ser,recém


##### Top-10 palavras associadas aos Top-3 Documentos de 'país'

In [204]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_03['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top3['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,brasil,traz
2,governo,presidente
3,nacional,oficial
4,paulo,mundo
5,mundo,mundial
6,grande,milhões
7,pessoas,jair
8,bolsonaro,durante
9,países,brasil
10,afirmou,bolsonaro


##### Top-10 palavras associadas aos Top-3 Documentos de 'pessoas'

In [205]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_04['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top4['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,então,brasil
2,país,bolsonaro
3,paulo,único
4,todo,twitter
5,deputado,três
6,federal,todos
7,dias,todo
8,disse,ter
9,onde,taxa
10,apenas,quase


##### Top-10 palavras associadas aos Top-3 Documentos de 'mulheres'

In [206]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_05['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top5['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,seguro,porque
2,forte,ministro
3,economista,ainda
4,aposentadoria,vai
5,igual,unidos
6,homens,todo
7,aprovada,texto
8,volume,ter
9,verdadeiramente,sobre
10,velhice,ser


#### Top-5 Documentos

In [207]:
k = 5
rank_d1 = top_docs(consultas[0], k)
rank_d2 = top_docs(consultas[1], k)
rank_d3 = top_docs(consultas[2], k)
rank_d4 = top_docs(consultas[3], k)
rank_d5 = top_docs(consultas[4], k)

In [208]:
tabela_top1 = cria_tabela_de_metricas(consultas[0], rank_d1, k)
tabela_top2 = cria_tabela_de_metricas(consultas[1], rank_d2, k)
tabela_top3 = cria_tabela_de_metricas(consultas[2], rank_d3, k)
tabela_top4 = cria_tabela_de_metricas(consultas[3], rank_d4, k)
tabela_top5 = cria_tabela_de_metricas(consultas[4], rank_d5, k)

##### Top-10 palavras associadas aos Top-5 Documentos de 'presidente'

In [209]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_01['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top1['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,país
2,disse,jair
3,brasil,brasil
4,jair,bolsonaro
5,governo,vez
6,feira,sobre
7,nesta,ser
8,contra,paulo
9,ser,ministério
10,sobre,haddad


##### Top-10 palavras associadas aos Top-5 Documentos de 'governo'

In [210]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_02['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top2['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,ser
2,presidente,parte
3,brasil,brasil
4,jair,todos
5,disse,todo
6,afirmou,situação
7,paulo,responsável
8,ainda,relação
9,país,primeiro
10,ser,presidente


##### Top-10 palavras associadas aos Top-5 Documentos de 'país'

In [211]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_03['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top3['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,brasil,brasil
2,governo,presidente
3,nacional,paulo
4,paulo,mundo
5,mundo,milhões
6,grande,governo
7,pessoas,durante
8,bolsonaro,bolsonaro
9,países,ainda
10,afirmou,vai


##### Top-10 palavras associadas aos Top-5 Documentos de 'pessoas'

In [212]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_04['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top4['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,então,ministro
2,país,federal
3,paulo,brasil
4,todo,bolsonaro
5,deputado,ainda
6,federal,três
7,dias,todo
8,disse,ter
9,onde,tempo
10,apenas,ser


##### Top-10 palavras associadas aos Top-5 Documentos de 'mulheres'

In [213]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_05['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top5['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,seguro,porque
2,forte,ministro
3,economista,ainda
4,aposentadoria,vai
5,igual,unidos
6,homens,todo
7,aprovada,texto
8,volume,ter
9,verdadeiramente,sobre
10,velhice,ser


#### Top-10 Documentos


In [214]:
k = 10
rank_d1 = top_docs(consultas[0], k)
rank_d2 = top_docs(consultas[1], k)
rank_d3 = top_docs(consultas[2], k)
rank_d4 = top_docs(consultas[3], k)
rank_d5 = top_docs(consultas[4], k)

In [215]:
tabela_top1 = cria_tabela_de_metricas(consultas[0], rank_d1, k)
tabela_top2 = cria_tabela_de_metricas(consultas[1], rank_d2, k)
tabela_top3 = cria_tabela_de_metricas(consultas[2], rank_d3, k)
tabela_top4 = cria_tabela_de_metricas(consultas[3], rank_d4, k)
tabela_top5 = cria_tabela_de_metricas(consultas[4], rank_d5, k)

##### Top-10 palavras associadas aos Top-10 Documentos de 'presidente'

In [216]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_01['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top1['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,país
2,disse,brasil
3,brasil,sobre
4,jair,lula
5,governo,disse
6,feira,contra
7,nesta,bolsonaro
8,contra,ano
9,ser,vez
10,sobre,ser


##### Top-10 palavras associadas aos Top-10 Documentos de 'governo'

In [217]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_02['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top2['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,brasil
2,presidente,disse
3,brasil,bolsonaro
4,jair,ser
5,disse,presidente
6,afirmou,política
7,paulo,paulo
8,ainda,jair
9,país,grande
10,ser,apenas


##### Top-10 palavras associadas aos Top-10 Documentos de 'país'

In [218]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_03['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top3['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,brasil,brasil
2,governo,sobre
3,nacional,presidente
4,paulo,unidos
5,mundo,paulo
6,grande,nacional
7,pessoas,feira
8,bolsonaro,estados
9,países,segundo
10,afirmou,grande


##### Top-10 palavras associadas aos Top-10 Documentos de 'pessoas'

In [219]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_04['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top4['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,então,ano
2,país,presidente
3,paulo,federal
4,todo,brasil
5,deputado,bolsonaro
6,federal,ainda
7,dias,ter
8,disse,ser
9,onde,porque
10,apenas,paulo


##### Top-10 palavras associadas aos Top-10 Documentos de 'mulheres'

In [220]:
tabela_DICE = pnd.DataFrame()
tabela_DICE['Dice before Term A Time'] = tabela_05['Dice']
tabela_DICE['Dice after Term A Time'] = tabela_top5['Dice']
tabela_DICE

,Dice before Term A Time,Dice after Term A Time
1,seguro,porque
2,forte,ministro
3,economista,ainda
4,aposentadoria,vai
5,igual,unidos
6,homens,todo
7,aprovada,texto
8,volume,ter
9,verdadeiramente,sobre
10,velhice,ser


#### Resultado:

De acordo com o que vamos aumentando o K, os resultados se mostram de maior qualidade, isso se da pelo fato de termos maior ligação entre as palavras. O top-k documentos torna bem mais rápido a computação que nos mostra as palavras que estão mais relacionadas aquela determinada consulta.